# compas_fea2 simple script approach

This example covers the analysis of a simple 2D frame structure generated completely from scratch. 

In [1]:
from compas_fea2.backends.abaqus.model import Model
from compas_fea2.backends.abaqus.model import Part
from compas_fea2.backends.abaqus.model import Node
from compas_fea2.backends.abaqus.model import ElasticIsotropic
from compas_fea2.backends.abaqus.model import BoxSection
from compas_fea2.backends.abaqus.model import BeamElement
from compas_fea2.backends.abaqus.model import Set

from compas_fea2.backends.abaqus.problem import Problem
from compas_fea2.backends.abaqus.problem import FixedDisplacement
from compas_fea2.backends.abaqus.problem import RollerDisplacementXZ
from compas_fea2.backends.abaqus.problem import PointLoad
from compas_fea2.backends.abaqus.problem import FieldOutput
from compas_fea2.backends.abaqus.problem import GeneralStaticStep

## Model generation

The fist step (of an FEA) is the generation of the analysis `model`. 
A model is always composed by:

1) geometry: nodes, connectivity among nodes, parts in which nodes and elements are groups, etc.
The geometry can either be created from scratch (as in this example) or be imported from other datastructures (networks, meshes, etc).

2) properties:  materials, sections, elements, interactions, etc. Properties are applied to the geometry to simulate the mechanical behaviour. 

In a compas workflow, geometry and properties are provided by the compas_package that creates the structure to analyse. 


In [2]:
model = Model(name='structural_model')

print(model)


compas_fea2 Model object
------------------------
name            : structural_model
# of parts      : 0
# of instances  : 0


### Parts

`Parts` are subregions of the model that can be considered to be independent from each other. However, similar parts of a model can still be modelled as different `Parts`. 

A good example of a `Part` could be a brick. 

In [3]:
model.add_part(Part(name='part-1'))

print(model.parts['part-1'])


compas_fea2 Part object
-----------------------
name            : part-1
# of nodes      : 0
# of elements   : 0


Nodes can be added to the `Part` or directly to the `Model` by specifying the name of the `Part` to which they belong.
In this example, we want to create a simple rectangular frame with a node on each side of the frame every 100mm.

In [4]:
for x in range(0, 1100, 100):
    model.add_node(Node(xyz=[x, 0.0, 0.0]), part='part-1')
for y in range(100, 600, 100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')
for x in range(900, -100, -100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')
for y in range(400, 0, -100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')

print(model.parts['part-1'].nodes[5])


compas_fea2 Node object
-----------------------
label      : n-5
key        : 5
x          : 500.0
y          : 0.0
z          : 0.0
ex         : None
ey         : None
ez         : None
mass       : None


We can now add the material properties we plan to use in our analysis. These material will then be available in the 'Model' and can be used in the definition of the structural sections.

In [5]:
# Define materials
model.add_material(ElasticIsotropic(name='mat_A', E=29000, v=0.17, p=2.5e-9))
model.add_material(ElasticIsotropic(name='mat_B', E=25000, v=0.17, p=2.5e-9))

print(model.materials['mat_A'])


compas_fea2 ElasticIsotropic object
-----------------------------------
name        : mat_A
E           : {'E': 29000}
v           : {'v': 0.17}
G           : {'G': 12393.162393162394}
p           : 2.5e-09


`compas_fea2` uses the same convention as Abaqus where the materials are assigned to the section and not to the elements (sections are assigned to the elements).
In the definition of a structural section you can specify a particular material by passing the material name. The structural properties are computed automatically accordingly to the type of section chosen (`BoxSection` in this case)

In [6]:
# Define sections
model.add_section(BoxSection(name='section_A', material='mat_A', a=20, b=80, t1=5, t2=5, t3=5, t4=5))
model.add_section(BoxSection(name='section_B', material='mat_B', a=20, b=80, t1=5, t2=5, t3=5, t4=5))

print(model.sections['section_A'])


compas_fea2 Section object
--------------------------
name  : section_A


Structural elements are defined by their type, connectivity and associated section. In this case we use a `BeamElement`, so we need to specify the 2 nodes it is connected to, with the 'BoxSection' defined before.
Elements can be added to the `model` one by one or as a list. 

In [7]:
# Generate elements between nodes
elements = []
for e in range(len(model.parts['part-1'].nodes)-1):
    elements.append((BeamElement(connectivity=[e, e+1], section='section_A')))
model.add_elements(elements=elements, part='part-1')
model.add_element(element=BeamElement(connectivity=[29, 0], section='section_B'), part='part-1')

print(model.parts['part-1'].elements[15])



compas_fea2 BeamElement object
------------------------------
key        : 15
etype      : beam
connectivity : [15, 16]


A `Set` is a unique collection of either nodes or elements and it is used to assign loads or boundary conditions.
In this case we defined two sets for the boundary conditions using node `0` and `10` and one for a point load at node `20`. 

In [8]:
# Define sets for boundary conditions and loads
model.add_assembly_set(Set(name='fixed', selection=[0], stype='nset'), instance='part-1-1')
model.add_assembly_set(Set(name='roller', selection=[10], stype='nset'), instance='part-1-1')
model.add_assembly_set(Set(name='pload', selection=[20], stype='nset'), instance='part-1-1')

In [9]:
print(model)


compas_fea2 Model object
------------------------
name            : structural_model
# of parts      : 1
# of instances  : 1


## Problem definition
The second part of the FEA is to define the `Problem` to be solved. To define a `Problem` you need a `Model`, boundary conditions (`bcs`), `laods`, field and history `outputs` and analysis 'steps'.  

In [10]:

# Create the Problem object
problem = Problem(name='test_structure', model=model)

# Assign boundary conditions to the node stes
problem.add_bcs(bcs=[RollerDisplacementXZ(name='bc_roller', bset='roller'),
                        FixedDisplacement(name='bc_fix', bset='fixed')])

# Assign a point load to the node set
problem.add_load(load=PointLoad(name='pload', lset='pload', y=-1000))

# Define the field outputs required
problem.add_field_output(fout=FieldOutput(name='fout'))

# Define the analysis step
problem.add_step(step=GeneralStaticStep(name='gstep', loads=['pload'], field_outputs=['fout']))

print(problem)



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
compas_fea2 Problem: test_structure
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Boundary Conditions
-------------------
  bc_roller : RollerDisplacementXZ
  bc_fix : FixedDisplacement

Loads
-----
  pload : PointLoad

Steps
-----
  fout : FieldOutputRequst

Steps Order
-----------
  standard : FieldOutputRequst

Field Output Requests
---------------------
  standard : FieldOutputRequst

History Output Requests
-----------------------
  standard : FieldOutputRequst




Once the `Probelm` is set, you can solve it running the analysis. You need to specify the location of the analysis files generated by Abaqus.
The output log for now is limited to the one automatically generated by Abaqus, but the plan is to expand it to have more info about the status of the analysis, possible warnings or errors, etc. 

In [11]:
# Solve the problem
# my_structure.write_input_file(path='C:/temp/test_structure')
problem.analyse(path='C:/temp/fea2_workshop')

***** Input file generated in: C:/temp/fea2_workshop\test_structure.inp *****

b'Analysis initiated from SIMULIA established products'
b'Abaqus JOB test_structure'
b'Abaqus 3DEXPERIENCE R2019x'
b'Begin Analysis Input File Processor'
b'1/8/2021 3:56:41 PM'
b'Run pre.exe'
b'1/8/2021 3:56:46 PM'
b'End Analysis Input File Processor'
b'Begin Abaqus/Standard Analysis'
b'1/8/2021 3:56:46 PM'
b'Run standard.exe'
b'1/8/2021 3:56:49 PM'
b'End Abaqus/Standard Analysis'
b'Begin Convert MFS to SFS'
b'1/8/2021 3:56:51 PM'
b'Run SMASimUtility.exe'
b'1/8/2021 3:56:51 PM'
b'End Convert MFS to SFS'
b'Abaqus JOB test_structure COMPLETED'
b''
b'Abaqus License Manager checked out the following licenses:\r\nAbaqus/Standard checked out 5 tokens from Flexnet server lic-abaqus.ethz.ch.\r\n<302 out of 320 licenses remain available>.\r\n'
***** Analysis successful - analysis time : 16.002654552459717 s *****


## Results Visualization

...not there yet... :)